<h1><center>Functional Brain Parcellation</center></h1>
<center><img src='stab_mat.png' class="img_title"></center>
<center>Pierre Bellec</center>
<center>University of Montreal, CRIUGM, CA</center>

<center>
    <div id="banner" style="overflow: hidden; justify-content:space-around;">
        <div class="" style="max-width: 20%;max-height: 20%;display: inline-block;">
            <img src="logo_udem.png" width="200px" alt="">
        </div>
        <div class="" style="max-width: 20%;max-height: 20%;display: inline-block;">
            <img src="logo_criugm.png" width="200px" alt="">
        </div>
        <div class="" style="max-width: 20%;max-height: 20%;display: inline-block;">
            <img src="logo_unf.png" width="200px" alt="">
        </div>
    </div>
</center>

<div class="row content">
    <div class="col-lg-8 col-md-8 col-xs-8">
        <img class="img_slide" src="varela2001.png" width="100%" alt="">
    </div>
    <div class="col-lg-4 col-md-4 col-xs-4">
        <h1><center>Why parcellations? understand brain organization</center></h1>
        <par>
            Local and distributed connectivity lead to the emergence of both local and distributed neuronal assemblies. From Varela et al., 2001.
        </par>
    </div>
</div>



<div class="row content">
    <div class="col-lg-4 col-md-4 col-xs-4">
        <h1><center>Why parcellations? reduce dimensionality</center></h1>
        <par>
            Parcels can be used as nodes to approximate brain networks as graphs. Here, the average connectome of the ADHD-200 sample is represented with several different parcellations, of varying resolutions. From Bellec et al., Neuroimage 2017.
        </par>
    </div>
    <div class="col-lg-8 col-md-8 col-xs-8">
        <img class="img_slide" src="graphs_adhd200.png" width="75%" alt="">
    </div>
</div>
